使用requests请求库
使用基本的正则表达式作为html解析工具

In [1]:
# urllib读取robots文件
from urllib.robotparser import RobotFileParser
rp = RobotFileParser()
rp.set_url('https://www.maoyan.com/robots.txt')
rp.read()
print(rp.can_fetch('*', 'https://www.maoyan.com/board/4'))

True


抓取第一页,跳过证书验证

检查网页源代码时，不选elements选项卡，选network选项卡，因为elements选项卡中的源码可能经过javascript操作与原始请求不同
从network选项卡查看网络原始请求得到的源码

In [128]:
import requests

#获取网页源码
def get_one_page(url):
    headers = {
    'Cookie': '__mta=142439949.1689062356247.1689066672133.1689066704116.17; uuid_n_v=v1; uuid=CD0172301FC011EEB5DB33031B0079F3AACF760FC52447248407EAA0249FD3F6; _csrf=da1f70aad5850525c5fa579f2bbb0f0153f1237ddbf41c2ef0159f289b4f682b; _lxsdk_cuid=18943f61ae8c8-001d3f80f60369-1b525634-13c680-18943f61ae9c8; _lxsdk=CD0172301FC011EEB5DB33031B0079F3AACF760FC52447248407EAA0249FD3F6; Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1689062351; Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1689066700; _lxsdk_s=189442b24ed-4d6-97e-153%7C%7C22',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Referer': 'https://tfz.maoyan.com/'
    }
    response = requests.get(url,headers=headers,verify=False)
    if(response.status_code==200):
        return response.text
    return None

In [129]:
import re

#解析源码
def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>',re.S)
    items = re.findall(pattern, html)
    for item in items:
    # yield生成器函数用于生成字典数据，但是不会改变item本身
        yield {'index': item[0],
            'image': item[1],
            'title': item[2].strip(),
            'actor': item[3].strip()[3:] if len(item[3]) > 3 else '',
            'time': item[4].strip()[5:] if len(item[4]) > 5 else '',
            'score': item[5].strip() + item[6].strip()}

In [130]:
def write_to_file(content): 
# with语句保证文件写操作之后，正确关闭文件
    with open('result.txt', 'a', encoding='utf-8') as f:    
        f.write(json.dumps(content, ensure_ascii=False)+'\n')

In [131]:
from requests.packages import urllib3
urllib3.disable_warnings()

爬取前10

In [132]:
url = 'https://www.maoyan.com/board/4?offset=0'  
html = get_one_page(url)  
print(html)
for item in parse_one_page(html):
    print(item)
    write_to_file(item)

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 9]><!--><html><!--<![endif]-->
<head>
  <title>TOP100榜 - 猫眼电影 - 一网打尽好电影</title>
  
  <link rel="dns-prefetch" href="//p0.meituan.net"  />
  <link rel="dns-prefetch" href="//p1.meituan.net"  />
  <link rel="dns-prefetch" href="//ms0.meituan.net" />
  <link rel="dns-prefetch" href="//ms1.meituan.net" />
  <link rel="dns-prefetch" href="//lx.meituan.net"/>
  <link rel="dns-prefetch" href="//lx1.meituan.net"/>
  <link rel="dns-prefetch" href="//plx.meituan.com"/>

  
  <meta charset="utf-8">
  <meta name="keywords" content="猫眼电影,电影排行榜,热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100">
  <meta name="description" content="猫眼电影热门榜单,包括热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100,多维度为用户进行选片决策">
  <meta http-equiv="cleartype" content="yes" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="renderer" content="webkit" />

  <meta name="HandheldFriendly" content="true" />
  <meta nam

爬取前100

In [133]:
import time 

for i in range(10):
    offset=i * 10
    url = 'http://maoyan.com/board/4?offset=' + str(offset)  
    html = get_one_page(url)  
    for item in parse_one_page(html):  
        print(item)  
        write_to_file(item)
    time.sleep(1)

{'index': '41', 'image': 'https://p0.pipi.cn/mmdb/d2dad5927e10fac7edf0ee42a9d44a4e0fc97.jpg?imageView2/1/w/160/h/220', 'title': '三块广告牌', 'actor': '弗兰西斯·麦克多蒙德,伍迪·哈里森,山姆·洛克威尔', 'time': '2018-03-02', 'score': '8.6'}
{'index': '42', 'image': 'https://p0.pipi.cn/mmdb/d2dad592c7e7e157e2395b12a043e8aba9dd4.jpg?imageView2/1/w/160/h/220', 'title': '末代皇帝', 'actor': '尊龙,陈冲,彼得·奥图尔', 'time': '1987-10-04(日本)', 'score': '8.8'}
{'index': '43', 'image': 'https://p0.pipi.cn/mmdb/d2dad59206dc69e19b7e12d3d395c0b36f1a9.jpg?imageView2/1/w/160/h/220', 'title': '心灵奇旅', 'actor': '杰米·福克斯,蒂娜·菲,菲利西亚·拉斯海德', 'time': '2020-12-25', 'score': '9.2'}
{'index': '44', 'image': 'https://p0.pipi.cn/mmdb/d2dad592c7e7e18ea367cbf41b1c2d9ec19bf.jpg?imageView2/1/w/160/h/220', 'title': '机器人总动员', 'actor': '本·贝尔特,艾丽莎·奈特,杰夫·格尔林', 'time': '2008-06-27(美国)', 'score': '9.3'}
{'index': '45', 'image': 'https://p0.pipi.cn/mmdb/d2dad592b122ff030cecd8d73ffdbf3dd00a6.jpg?imageView2/1/w/160/h/220', 'title': '超脱', 'actor': '阿德里安·布劳迪,马西娅·盖伊·哈登,詹